# Create Triples

A short guide to adding triples to the Golden decentralized knowledge graph using the godel SDK. Please see the [Golden Protocol FAQ Guide](https://www.notion.so/goldenhq/Golden-Protocol-FAQ-78ae2357b9af44aeaa655cb1b1966ee4) and the [Adding Structured Data Guide](https://www.notion.so/goldenhq/Adding-Structured-Data-Guide-ae657337bf4f4e54ae4402df083c76ac) to learn more about Golden, data submission, and rewards.

Note: Attribution and eligibility for testnet points on triple submissions will be assigned by the earliest timestamped transaction.

## Prerequisite

Godel [Authentication](https://docs.golden.xyz/api/godel-python-sdk/authentication)

This guide requires you install Godel's data-tools.

You can do this with `pip install godel` and comes pre-installed if using the godel docker image.

### 1. Connect to Golden Web3 API

Let's connect the python wrapper to the Golden GraphQL API.

Make sure you ran through the prerequisites for this guide and have learned to authenticate and retrieve your JWT token in Godel.

In [ ]:
from godel import GoldenAPI

JWT_TOKEN = #YOUR_JWT_TOKEN_HERE
API_URL = "https://dapp.golden.xyz/graphql"
SANDBOX_URL = "https://sandbox.dapp.golden.xyz/graphql" # Use the sandbox API to test your submissions
goldapi = GoldenAPI(url=API_URL)
goldapi.set_jwt_token(jwt_token=JWT_TOKEN)

Test that you can hit the API with `entity_search()`, and we'll save the results so we can use the resulting entity as our subject entity for this guide. We are using pandas here to make working with data easier, but it is not a requirement.

In [ ]:
import pandas as pd

# Test with search
search_results = goldapi.entity_search(name="Miles")
search_results_df = pd.DataFrame(search_results["data"]["entityByName"]["nodes"])
search_results_df

### 2. Get Predicates

You can run the code below to get the list of accepted predicates available for the protocol.

In [ ]:
import pandas as pd

predicates = {}
for p in goldapi.predicates()["data"]["predicates"]["edges"]:
    p = p["node"]
    predicates[p["name"]] = {"id": p["id"], "objectType": p["objectType"]} 
predicates_df = pd.DataFrame(predicates).transpose()
predicates_df.head()

### 3. Create Triple

First, view the `CreateStatementInput` object schema.

In [ ]:
from godel.schema import CreateStatementInput, StatementInputRecordInput

CreateStatementInput

Now you can input your triple data. Make sure to check the predicate object type to ensure your object entity ID or object value matches the object type.

#### Triple with Object Value Example

In [ ]:
# Triple data
subject_id = search_results_df["id"][0]  # Miles Wolff
predicate_id = predicates_df["id"][1]  # Email Address
object_value = "guide@example.com"

# Create statement input API
create_statement_input = CreateStatementInput(
    subject_id=subject_id,
    predicate_id=predicate_id,
    object_value=object_value,
    citation_urls=["https://golden.com"],
)

### WARNING: Running the code below may charge gas fees and stake testnet points with your wallet. You may lose testnet points by submitting incorrect data.

In [ ]:
data = goldapi.create_statement(create_statement_input=create_statement_input)
data

#### Triple with Object Entity ID Example

In [ ]:
# Triple data
subject_id = search_results_df["id"][0]  # Miles Wolff
predicate_id = "e4f94b98-c56a-4bd2-a9fd-5fd11603e7e8"  # CEO Of predicate
object_entity_id = "20ab9281-fd5f-4717-ab73-ecd24fff66fe"  # Huel and Sons Entity ID

# Create statement input API
create_statement_input = CreateStatementInput(
    subject_id=subject_id,
    predicate_id=predicate_id,
    object_entity_id=object_entity_id,
    citation_urls=["https://golden.com"],
)

### WARNING: Running the code below may charge gas fees and stake testnet points with your wallet. You may lose testnet points by submitting incorrect data.

In [ ]:
data = goldapi.create_statement(create_statement_input=create_statement_input)
data